# Scraping LADOT Volume Data from PDFs, Part 2

##### Where I Left Off
In the first python notebook, I described the problem and general approach I was taking to solve it. I downloaded all the manual count PDF documents and setup the file tables for later.

## PDF Text Extraction Process
As I discussed in the first python notebook, my process for extracting the text into usable data can be broken down into the few key parts: (1) define bounding boxes, (2) search for text within the bounding boxes (3) reformat the resulting text into multiple data tables, and (4) join the resulting tables to the ID established by the Bureau of Engineering. 

##### Define Bounding Boxes
This was tricky. I initially began defining bounding boxes using pixel measurements from a few sample pages. However, I quickly realized that due to the second of the challenges I mentioned above that this would not work, since the tables are in different locations among the PDF documents. 

Instead, I decided to create bounding boxes on the fly for each document using relative positioning of certain keywords that appeared almost always on each PDF document. Using pdfquery, I could begin by searching the document for these keywords and then extract the x,y pixel coordinate locations for the bounding box of each one. By getting the coordinates of multiple keyword objects on the page, I could construct a set of bounding boxes that seemed to perform relatively well in capturing data tables.

(create image of what this looked like)

##### Search for Text within Bounding Boxes
Once I had the coordinates of the bounding boxes, this part was quite easy, using PDFQuery to extract text

(do i need to adjust any of the parameters in rapidminer??)

##### Reformat the Resulting Text into Data Tables
The final problem included taking the scraped text from the bounding boxes and reformatting them into usable data tables. I kept in mind the relational database model as I set the format for these tables. From the PDF image above, I decided on the following tables and attributes:

*tbl_manualcount_info:* This table contains the basic information about the manual count summary. Each count will have one tuple with the following information:
* street_ns: The North / South Street running through the intersection
* street_ew: The East / West street running through the intersection
* dayofweek: Day of the Week
* date: Date, in datetime format
* weather: Prevailing weather at the time of the count (Clear, Sunny, etc.)
* hours: the hours of the count (text)
* school_day: A Yes / No indication of whether the count occurred on a school day. This is important because it heavily affects the volume counts
* int_code: The "I/S Code" on the form corresponds to the CL_Node_ID on the BOE Centerline. This ID field makes it easy to join to the City's centerline network
* district: The DOT field district in which the count took place
* count_id: Unique identifier assigned to the summary

*tbl_manualcount_dualwheel:* This table contains count data for dual-wheeled (motorcycles), bikes, and buses. Each form will have 12 tuples with the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Intersection approach being measured (N,S,E,W)
* type: Dual-Wheeled / Bikes / Buses
* volume: Count

*tbl_manualcount_peak:* This table contains the peak hour / 15 minute counts. Each form will have 16 tuples with the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Intersection approach being measured (N,S,E,W)
* type: The type of count
    * am_15: The AM peak 15 minute count
    * am_hour: The AM peak hour count
    * pm_15: The PM peak 15 minute count
    * pm_hour: The PM peak hour count
* time: Time of each count (in datetime format)
* volume: Count

*tbl_manualcount_volumes:* This table contains the main volume counts for each approach at the intersection. The number of tuples for each form will vary depending on the number of hours surveyed. A 6-hour count will have 6 hours * 3 directions (left, right, through) * 4 approach directions = 54 tuples. Each tuple will have the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Northbound (NB) / Southbound (SB) / Eastbound (EB) / Westbound (WB)
* movement: Right-Turn (Rt) / Through (Th) / Left-Turn (Lt)
* start_time: Start time of that count, in datetime format
* end_time: End time of that count, in datetime format
* volume: Count

*tbl_manualcount_peds:* This table contains pedestrian and schoolchildren counts during the same time as the main volume counts, so the number of tuples will also be dependent on the number of hours the location was surveyed. Each tuple will have the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* xing_leg: The leg of the intersection that is being crossed. South Leg (SL) / North Leg (NL) / West Leg (WL) / East Leg (EL)
* type: Ped / Sch
* start_time: Start time of that count, in datetime format
* end_time: End time of that count, in datetime format
* volume: Count


In [2]:
### Setup
import csv
import glob
from datetime import datetime, date, time
import pdfquery
import pandas as pd
import numpy as np
import os
from pathlib2 import Path

### Step 1: Prepare the Data Tables
Based on the outline above, the first step is to prepare dataframes for each of the volume-related tables discussed above. Once I have the dataframe table structures, the next step involves looping through all the manual count PDFs, running my text extract function, and then inserting the rows into the appropriate tables. 

In [3]:
import pandas as pd

### Load NavLA Count file table
count_files_df = pd.read_csv('data/TrafficCountFileStructure/navLAfiles.csv',index_col=0)
manual_count_files_df = count_files_df[(count_files_df['type'] == 'manual')].reset_index()

### Create Dataframes for each output table
# tbl_manualcount_info
info_columns = ['street_ns','street_ew','dayofweek','date','weather','hours','school_day','int_code','district','count_id']
tbl_manualcount_info = pd.DataFrame(columns=info_columns)

# tbl_manualcount_dualwheel
dualwheel_columns = ['count_id','approach','type','volume']
tbl_manualcount_dualwheel = pd.DataFrame(columns=dualwheel_columns)

# tbl_manualcount_peak
peak_columns = ['count_id','approach','type','time','volume']
tbl_manualcount_peak = pd.DataFrame(columns=peak_columns)

# tbl_manualcount_volumes
vol_columns = ['count_id','approach','movement','start_time','end_time','volume']
tbl_manualcount_volumes = pd.DataFrame(columns=vol_columns)

# tbl_manualcount_peds
ped_columns = ['count_id','xing_leg','type','start_time','end_time','volume']
tbl_manualcount_peds = pd.DataFrame(columns=ped_columns)

manual_count_files_df.head()

,count_id,cl_node_id,location,traffic_id,type,file
0,1,3667,ISLAND AVE at L ST,1,manual,3667_ISLLST94.PDF
1,2,3680,FIGUEROA ST at L ST,2,manual,3680_FIGLST94.PDF
2,3,3727,FRIES AVE at HARRY BRIDGES BLVD,3,manual,3727_FRIHAR95.PDF
3,4,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,3787_ANAFAR100817.PDF
4,5,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,FARRAGUT.ANAHEIM.110119-MAN.PDF


### Step 2: Run the PDF text scraping script (extract / transform)
The powerhouse behind this process is the script I built to read a PDF and extract text into a list of dictionaries (one for each table). Because this script is so long, I opted not to include the code in the notebook, and instead treated it as a module that I imported using "import VolumeCountSheets_V2" above. I call the "pdf_extract" function from the module on each PDF.

### Step 3: Append each resulting dictionary to the Pandas dataframe (load)
The function returns a list of dictionaries, one for each dataframe. I then take the pandas dataframes I constructed in the cell above and append each dictionary to the appropriate one.

In [5]:
# Import PDF Scrape Module
import VolumeCountSheets_V2

# Setup Counter of sucessful / failed attempts
success = 0
failures = 0

# Create empty lists for storing count data
manualcount_volumes = []
ped_volumes = []
peak_volumes = []
info = []
specveh_volumes = []

#### Offset values for counters
# Offset dictionary, with offset values for all types
Offsets = {}

# Type 1: NDS1
Offsets['NDS1'] = {}
Offsets['NDS1']['North/South'] = [70,-5,350,15]
Offsets['NDS1']['East/West'] = [70,-5,350,15]
Offsets['NDS1']['Day:'] = [5,-5,120,15]
Offsets['NDS1']['Date:'] = [30,-8,130,15]
Offsets['NDS1']['Weather:'] = [30,-8,130,15]
Offsets['NDS1']['Hours:'] = [30,-8,140,15]
Offsets['NDS1']['School Day:'] = [40,-5,120,15]
Offsets['NDS1']['District:'] = [40,-5,130,15]
Offsets['NDS1']['WHEELED'] = [70,-30,380,12]
Offsets['NDS1']['N/B TIME'] = {}
Offsets['NDS1']['S/B TIME'] = {}
Offsets['NDS1']['E/B TIME'] = {}
Offsets['NDS1']['W/B TIME'] = {}
Offsets['NDS1']['N/B TIME']['vol'] = [-5,-91,18,-1]
Offsets['NDS1']['N/B TIME']['time'] = [18,-91,55,-1]
Offsets['NDS1']['S/B TIME']['vol'] = [-5,-91,18,-1]
Offsets['NDS1']['S/B TIME']['time'] = [18,-91,55,-1]
Offsets['NDS1']['E/B TIME']['vol'] = [-5,-91,18,-1]
Offsets['NDS1']['E/B TIME']['time'] = [18,-91,55,-1]
Offsets['NDS1']['W/B TIME']['vol'] = [-5,-91,18,-1]
Offsets['NDS1']['W/B TIME']['time'] = [18,-91,55,-1]
Offsets['NDS1']['survey_hours'] = [-2,-101,23,-15] # These offset values are based on the lower left corner of 'NORTHBOUND Approach'
Offsets['NDS1']['NORTHBOUND Approach'] = [43,-95,135,-19]
Offsets['NDS1']['SOUTHBOUND Approach'] = [43,-95,135,-19]
Offsets['NDS1']['EASTBOUND Approach'] = [43,-95,135,-19]
Offsets['NDS1']['WESTBOUND Approach'] = [43,-95,135,-19]
Offsets['NDS1']['XING S/L'] = [-5,-85,45,-15]
Offsets['NDS1']['XING N/L'] = [-5,-85,45,-15]
Offsets['NDS1']['XING W/L'] = [-5,-85,45,-15]
Offsets['NDS1']['XING E/L'] = [-5,-85,45,-15]

# Type 3: Rev92_1
# Remove IS Code label for this version ('I/S CODE' field was not present on this form)

#### Loop through files, Run function
for index, row in manual_count_files_df.iterrows():
    

    print(row['file'])
    if index > 1:
        break
    
    
    # Folder location of all the count PDFs
    #fileloc = 'data/TrafficCountData/Manual/All/' + row['file']
    #fileloc = 'data/TrafficCountData/Manual/All/' + 'FARRAGUT.ANAHEIM.110119-MAN.PDF'
    fileloc = 'data/TrafficCountData/Manual/All/' + '4009_HOO16893.pdf'
    
    # ID for the count
    count_id = row['count_id']
    if index%100 == 0:
        print("Current Count:" + str(index+1))

    try:
        if Path(fileloc).exists():
            
            # Run the extract function
            Manual_TC = VolumeCountSheets_V2.pdf_extract(fileloc, Offsets['NDS1'])
                             
            # Append each row to our lists
            for m in Manual_TC['Spec_Veh']:
                m['count_id'] = str(count_id)
                specveh_volumes.append(m)
            
            for j in Manual_TC['Volume']:
                j['count_id'] = str(count_id)
                manualcount_volumes.append(j)
                
            for k in Manual_TC['Pedestrian']:
                k['count_id'] = str(count_id)
                ped_volumes.append(k)
                
            for l in Manual_TC['PeakVol']:
                l['count_id'] = str(count_id)
                peak_volumes.append(l)
                
            Manual_TC['Info']['count_id'] = str(count_id)
            info.append(Manual_TC['Info'])
            
            # Update success counter
            success+=1
            print('success')
            
                
        else:
            failures += 1
            print("path does not exist")
        
    except:
        raise
        failures += 1


# Create dataframes from lists of dictionaries
info_df = pd.DataFrame.from_records(info)
manualcount_df = pd.DataFrame.from_records(manualcount_volumes)
pedestrian_df = pd.DataFrame.from_records(ped_volumes)
peakvol_df = pd.DataFrame.from_records(peak_volumes)
specveh_df = pd.DataFrame.from_records(specveh_volumes)

# Save dataframes to CSVs
manualcount_df.to_csv(path_or_buf='data/TrafficCountData/Results/manualcount_test.csv',sep=',')
pedestrian_df.to_csv(path_or_buf='data/TrafficCountData/Results/pedestrian_test.csv',sep=',')
peakvol_df.to_csv(path_or_buf='data/TrafficCountData/Results/peakvol_test.csv',sep=',')
specveh_df.to_csv(path_or_buf='data/TrafficCountData/Results/SpecialVehicle_test.csv',sep=',')
info_df.to_csv(path_or_buf='data/TrafficCountData/Results/info_test.csv',sep=',')

# Print the dataframes here
print("Success Count")
print(str(success))
print("Failure Count")
print(failures)


3667_ISLLST94.PDF
Current Count:1


TypeError: float() argument must be a string or a number, not 'NoneType'